From the main directory "Ontolearn", run the commands for NCES data mentioned [here](https://ontolearn-docs-dice-group.netlify.app/usage/02_installation#download-external-files) to download pretrained models and datasets.

## Inference with NCES

In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from ontolearn.metrics import F1
from owlapy.parser import DLSyntaxParser



/home/nkouagou/.conda/envs/onto/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", quality_func=F1(), learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", load_pretrained=True, max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32, pretrained_model_name="SetTransformer")



 Loaded pretrained model! 



In [3]:
KB = KnowledgeBase(path=nces.knowledge_base_path)

In [4]:
dl_parser = DLSyntaxParser(nces.kb_namespace)

### Let's learn the concept ``Father''

In [5]:
father = dl_parser.parse('Father')

In [6]:
not_father = dl_parser.parse('¬Father') # For negative examples

In [7]:
pos = set([ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(father)][:10])

In [8]:
neg = set([ind.get_iri().as_str().split("/")[-1] for ind in KB.individuals(not_father)])

#### Prediction with SetTransformer

In [9]:
node = nces.fit(pos, neg).best_hypotheses()

In [10]:
node

<class 'ontolearn.search.NCESNode'> at 0xb7e165c	Father ⊓ (∃ married.(Father ⊓ (∀ hasParent.(¬Sister))))	Quality:0	Length:10	|Indv.|:0

In [10]:
nces.sorted_examples

False

#### Prediction with GRU

In [11]:
nces.pretrained_model_name = 'GRU'
nces.refresh()
node = nces.fit(pos, neg).best_hypotheses()



 Loaded pretrained model! 






In [12]:
node

<class 'ontolearn.search.NCESNode'> at 0xf897970	Father ⊔ Daughter	Quality:0.27778	Length:3	|Indv.|:112

#### Prediction with LSTM

In [13]:
nces.pretrained_model_name = 'LSTM'
nces.refresh()
node = nces.fit(pos, neg).best_hypotheses()



 Loaded pretrained model! 






In [14]:
node

<class 'ontolearn.search.NCESNode'> at 0xc8d04ac	Grandfather ⊔ (∃ hasChild.(Female ⊓ (∃ married.(¬Parent))))	Quality:0.48485	Length:10	|Indv.|:58

#### Prediction with ensemble SetTransformer+GRU+LSTM

In [15]:
nces.pretrained_model_name = ['SetTransformer','GRU','LSTM']
nces.refresh()
node = nces.fit(pos, neg).best_hypotheses()



 Loaded pretrained model! 



 Loaded pretrained model! 



 Loaded pretrained model! 






In [16]:
node

<class 'ontolearn.search.NCESNode'> at 0xf8c4284	Father ⊓ (∃ married.(Mother ⊓ (∃ hasParent.(¬Daughter))))	Quality:0.75	Length:10	|Indv.|:25

### Scalability of NCES (solving multiple learning problems in a go!)

In [17]:
import json, time

In [18]:
with open("../NCESData/family/training_data/Data.json") as file:
    data = list(json.load(file).items())
## The function below takes an iterable of tuples
LPs = list(map(lambda x: (x[0], x[1]["positive examples"], x[1]["negative examples"]), data))

In [19]:
## We solve 256 learning problems!
lps = LPs[:256]

In [20]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False) ## Now predict with nces.fit_from_iterable.
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")




Duration:  0.9786672592163086 seconds!


In [21]:
print(concepts[0])

OWLClass(IRI('http://www.benchmark.org/family#','Sister'))


In [20]:
print(len(concepts))

256


### Change pretrained model name, e.g., use ensemble model prediction

In [21]:
nces.pretrained_model_name = ['SetTransformer', 'GRU']

In [22]:
nces.pretrained_model_name

['SetTransformer', 'GRU']

In [23]:
nces.refresh()



 Loaded pretrained model! 



 Loaded pretrained model! 



In [24]:
t0 = time.time()
concepts = nces.fit_from_iterable(lps, verbose=False)
t1 = time.time()
print("Duration: ", t1-t0, "seconds!")

Duration:  8.768116474151611 seconds!


In [25]:
concepts[-1]

OWLObjectAllValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLObjectComplementOf(OWLClass(IRI('http://www.benchmark.org/family#','PersonWithASibling'))))

## Training NCES

In [26]:
import json, time
with open("../NCESData/family/training_data/Data.json") as file:
    data = json.load(file) # Training data. Below we use the first 200 datapoints to train the synthesizer. You train on the full data, e.g. on GPU by remobing "[:200]"

In [27]:
from ontolearn.concept_learner import NCES

In [28]:
nces = NCES(knowledge_base_path="../NCESData/family/family.owl", learner_name="SetTransformer",
     path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv", max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32,
            load_pretrained=False, pretrained_model_name="SetTransformer")

In [29]:
nces.train(list(data.items())[-200:], epochs=20, learning_rate=0.001, save_model=True, storage_path=f"./NCES-{time.time()}/")

********************Trainable model size********************
Synthesizer:  515296
********************Trainable model size********************

Training on CPU, it may take long...

##################################################

SetTransformer starts training... 

################################################## 



Loss: 0.5712, Soft Acc: 74.01%, Hard Acc: 71.02%: 100%|██████████| 20/20 [01:12<00:00,  3.63s/it]

Top performance: loss: 0.5712, soft accuracy: 74.65% ... hard accuracy: 72.23%

SetTransformer saved
